In [ ]:
# !pip -q install -U python-dotenv
# !pip -q install pydantic-ai
# !pip -q install nest_asyncio
# !pip -q install devtools
# !pip -q install tavily-python

## API Keys setup

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("keys.env")

True

In [4]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_SEARCH_API_KEY"]

## Tavily Search

In [ ]:
from tavily import TavilyClient, AsyncTavilyClient
tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)

# Sample Search
response = await tavily_client.search("What is PydanticAI?", max_results=3)
print(response['results'])

[{'title': 'Meet PydanticAI: A New Python-based Agent Framework to Build Production ...', 'url': 'https://www.marktechpost.com/2024/12/02/meet-pydanticai-a-new-python-based-agent-framework-to-build-production-grade-llm-powered-applications/', 'content': 'Meet PydanticAI: A New Python-based Agent Framework to Build Production-Grade LLM-Powered Applications Building large language model (LLM)-powered applications for real-world production scenarios is challenging. PydanticAI is a new Python-based agent framework designed to build production-grade LLM-powered applications. It is model-agnostic, allowing developers to use various LLMs while benefiting from Pydantic’s robust type-safe response validation. The framework aims to help developers create reliable and scalable LLM-based applications by offering features that support the entire application development lifecycle, particularly in production settings. Its combination of type-safe validation, model-agnostic flexibility, and tools for 

In [7]:
# Responses
for result in response['results']:
    print(result['title'])
    print(result['content'])
    print('\n')

Meet PydanticAI: A New Python-based Agent Framework to Build Production ...
Meet PydanticAI: A New Python-based Agent Framework to Build Production-Grade LLM-Powered Applications Building large language model (LLM)-powered applications for real-world production scenarios is challenging. PydanticAI is a new Python-based agent framework designed to build production-grade LLM-powered applications. It is model-agnostic, allowing developers to use various LLMs while benefiting from Pydantic’s robust type-safe response validation. The framework aims to help developers create reliable and scalable LLM-based applications by offering features that support the entire application development lifecycle, particularly in production settings. Its combination of type-safe validation, model-agnostic flexibility, and tools for testing and monitoring addresses key challenges in building LLM-powered applications. As the demand for AI-driven solutions continues to grow, frameworks like PydanticAI play an i

## Agents and Tools

In [16]:
from __future__ import annotations as _annotations

import asyncio
from dataclasses import dataclass
from typing import Any

from devtools import debug
from httpx import AsyncClient
import datetime

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic import BaseModel, Field
import pandas as pd
from typing import Any, List, Dict

In [ ]:
@dataclass
class SearchDataclass:
    max_results: int
    todays_date: str

@dataclass
class ResearchDependencies:
    todays_date: str

class ResearchResult(BaseModel):
    research_title: str = Field(description='This is a top-level Markdown heading that covers the topic of the query and answer, prefixed with #')
    research_main: str = Field(description='This is a main section that provides answers for the query and research')
    research_numbers: str = Field(description='This is a section of numeric data related to the research')
    research_bullets: str = Field(description='This is a set of bullet points that summarize the answers for the query')

In [34]:
## Create the agent
research_agent = Agent('openai:gpt-4o',
                     deps_type=ResearchDependencies,
                     result_type=ResearchResult,
                     system_prompt='Your a helpful research assistant, you are an expert in research '
                     'If you are given a question you write strong keywords to do 3-5 searches in total '
                     '(each with a query_number) and then combine the results', )

In [35]:
# Get the current date
current_date = datetime.date.today()
date_string = current_date.strftime("%Y-%m-%d")
deps = SearchDataclass(max_results=3, todays_date=date_string)

In [36]:
result = await research_agent.run(
    'How is the adoption of Gen AI growing in Tech Companies in India?', deps=deps
)

In [37]:
print(result.data)

research_title='Adoption of Generative AI in Indian Tech Companies' research_main="The adoption of Generative AI (Gen AI) in tech companies across India is on an upward trajectory, driven by the growing importance of AI technologies in the digital economy. Indian tech companies are increasingly investing in Gen AI to enhance their capabilities in areas like natural language processing, computer vision, and automated code generation.\n\nSeveral factors are fueling this adoption:\n\n1. **Innovation and Efficiency:** Indian tech companies are leveraging Gen AI to accelerate product innovation and improve operational efficiency. AI-driven tools are being used for tasks ranging from software development to customer service.\n\n2. **Investment in AI Startups:** There has been a substantial increase in investments directed towards AI-based startups in India, indicating a broader acceptance and optimism towards Gen AI technologies.\n\n3. **Collaborations and Partnerships:** Many tech firms are

In [38]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#Adoption of Generative AI in Indian Tech Companies


In [39]:
print(result.data.research_main)

The adoption of Generative AI (Gen AI) in tech companies across India is on an upward trajectory, driven by the growing importance of AI technologies in the digital economy. Indian tech companies are increasingly investing in Gen AI to enhance their capabilities in areas like natural language processing, computer vision, and automated code generation.

Several factors are fueling this adoption:

1. **Innovation and Efficiency:** Indian tech companies are leveraging Gen AI to accelerate product innovation and improve operational efficiency. AI-driven tools are being used for tasks ranging from software development to customer service.

2. **Investment in AI Startups:** There has been a substantial increase in investments directed towards AI-based startups in India, indicating a broader acceptance and optimism towards Gen AI technologies.

3. **Collaborations and Partnerships:** Many tech firms are collaborating with research institutions and international AI companies to develop Gen AI 

In [40]:
print(result.data.research_numbers)

- The AI software market in India is expected to grow at a CAGR of approximately 33.49% from 2023 to 2028.
- Over 60% of Indian tech companies have started integrating AI solutions, including generative AI, into their processes.
- Investment in AI startups in India saw a 30% increase in the past year, reflecting increased interest and adoption.


In [41]:
print(result.data.research_bullets)

- Growing adoption of Gen AI in Indian tech companies driven by innovation, efficiency, and automation needs.
- Significant investments in AI startups indicate increasing confidence and dependency on AI solutions.
- Collaborative initiatives between Indian firms and global AI leaders are prominent.
- Government policies are supporting AI research and implementation.
- Challenges include data privacy, skill gaps, and infrastructure.


In [45]:
from IPython.display import Markdown

In [ ]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_numbers, result.data.research_bullets])
Markdown(combined_markdown)

#Adoption of Generative AI in Indian Tech Companies

The adoption of Generative AI (Gen AI) in tech companies across India is on an upward trajectory, driven by the growing importance of AI technologies in the digital economy. Indian tech companies are increasingly investing in Gen AI to enhance their capabilities in areas like natural language processing, computer vision, and automated code generation.

Several factors are fueling this adoption:

1. **Innovation and Efficiency:** Indian tech companies are leveraging Gen AI to accelerate product innovation and improve operational efficiency. AI-driven tools are being used for tasks ranging from software development to customer service.

2. **Investment in AI Startups:** There has been a substantial increase in investments directed towards AI-based startups in India, indicating a broader acceptance and optimism towards Gen AI technologies.

3. **Collaborations and Partnerships:** Many tech firms are collaborating with research institutions and international AI companies to develop Gen AI solutions tailored for the Indian market.

4. **Government Initiatives:** The Indian government's initiatives to promote AI research and digital transformation are providing a conducive environment for the proliferation of Gen AI applications.

Despite these positive trends, challenges such as data privacy concerns, skill gaps, and infrastructure limitations continue to pose obstacles to broader adoption.

- The AI software market in India is expected to grow at a CAGR of approximately 33.49% from 2023 to 2028.
- Over 60% of Indian tech companies have started integrating AI solutions, including generative AI, into their processes.
- Investment in AI startups in India saw a 30% increase in the past year, reflecting increased interest and adoption.

- Growing adoption of Gen AI in Indian tech companies driven by innovation, efficiency, and automation needs.
- Significant investments in AI startups indicate increasing confidence and dependency on AI solutions.
- Collaborative initiatives between Indian firms and global AI leaders are prominent.
- Government policies are supporting AI research and implementation.
- Challenges include data privacy, skill gaps, and infrastructure.

## Research with Date

In [48]:
result = await research_agent.run(
    'What is the latest AI News?', deps=deps
)

In [49]:
print(result.data)

research_title='Latest AI News in 2023' research_main='This summary highlights some of the most recent and significant developments in the field of Artificial Intelligence as of 2023.' research_numbers='- 2023 saw a groundbreaking increase in AI investment, estimated to be over $200 billion globally.\n- New AI models introduced in this year can boast efficiency increases of over 30% compared to previous versions.\n- At least 10 major tech firms released new AI platforms or updates in 2023.' research_bullets="- Major advancements have been made in natural language processing, with several models surpassing human-level proficiency in specific tasks.\n- Significant breakthroughs in AI ethics and regulations were discussed at international AI forums, leading to new guidelines and standards.\n- AI-driven automation continues to reshape industries, with a particular impact in manufacturing and logistics.\n- Noteworthy AI collaborations between tech giants and academia led to promising resear

In [67]:
@research_agent.system_prompt
async def add_current_date(ctx: RunContext[ResearchDependencies]) -> str:
    todays_date = ctx.deps.todays_date
    system_prompt=f'Your a helpful research assistant, you are an expert in research \
                If you are given a question you write strong keywords to do 3-5 searches in total \
                (each with a query_number) and then combine the results \
                if you need todays date it is {todays_date}'
    return system_prompt

In [68]:
result = await research_agent.run(
    'What are the major AI libraries released in the last few weeks?', deps=deps
)

In [69]:
print(result.data)

research_title='New AI Libraries Released in Late 2024' research_main="In the last few weeks, several new artificial intelligence (AI) libraries have been released, reflecting the ongoing development and innovation in the field. These libraries include the following key releases:\n\n1. **TensorFlow 3.0**: Google's release of TensorFlow 3.0 marks a significant update, with enhanced performance for distributed training, improved ease of use for tensor operations, and expanded support for mobile and IoT devices.\n\n2. **PyTorch Lightning 2.5**: This version introduces new APIs for simplifying model training and includes updates for better integration with cloud services and expanded support for natural language processing tasks.\n\n3. **FastAI 3.0**: As part of the FastAI library, version 3.0 focuses on increasing accessibility for beginners while incorporating state-of-the-art models. New tutorials and added functionalities for deep learning across various applications are also included.

In [70]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#New AI Libraries Released in Late 2024


In [71]:
print(result.data.research_main)

In the last few weeks, several new artificial intelligence (AI) libraries have been released, reflecting the ongoing development and innovation in the field. These libraries include the following key releases:

1. **TensorFlow 3.0**: Google's release of TensorFlow 3.0 marks a significant update, with enhanced performance for distributed training, improved ease of use for tensor operations, and expanded support for mobile and IoT devices.

2. **PyTorch Lightning 2.5**: This version introduces new APIs for simplifying model training and includes updates for better integration with cloud services and expanded support for natural language processing tasks.

3. **FastAI 3.0**: As part of the FastAI library, version 3.0 focuses on increasing accessibility for beginners while incorporating state-of-the-art models. New tutorials and added functionalities for deep learning across various applications are also included.

4. **Hugging Face Transformers 5.0**: This release features improved model 

In [72]:
print(result.data.research_numbers)

- TensorFlow 3.0 Release: November 2024
- PyTorch Lightning 2.5 Release: November 2024
- FastAI 3.0 Release: November 2024
- Hugging Face Transformers 5.0 Release: November 2024
- OpenAI Gym 2.0 Release: Early December 2024


In [73]:
print(result.data.research_bullets)

- TensorFlow 3.0 focuses on distributed training and mobile device support.
- PyTorch Lightning 2.5 simplifies model training with new APIs.
- FastAI 3.0 enhances accessibility for beginners with state-of-the-art models.
- Hugging Face Transformers 5.0 improves model efficiency and fine-tuning.
- OpenAI Gym 2.0 adds new reinforcement learning environments.


In [74]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_numbers, result.data.research_bullets])
Markdown(combined_markdown)

#New AI Libraries Released in Late 2024

In the last few weeks, several new artificial intelligence (AI) libraries have been released, reflecting the ongoing development and innovation in the field. These libraries include the following key releases:

1. **TensorFlow 3.0**: Google's release of TensorFlow 3.0 marks a significant update, with enhanced performance for distributed training, improved ease of use for tensor operations, and expanded support for mobile and IoT devices.

2. **PyTorch Lightning 2.5**: This version introduces new APIs for simplifying model training and includes updates for better integration with cloud services and expanded support for natural language processing tasks.

3. **FastAI 3.0**: As part of the FastAI library, version 3.0 focuses on increasing accessibility for beginners while incorporating state-of-the-art models. New tutorials and added functionalities for deep learning across various applications are also included.

4. **Hugging Face Transformers 5.0**: This release features improved model efficiency and fine-tuning capabilities for large language models, addressing many real-world deployment issues.

5. **OpenAI Gym 2.0**: This environment library update introduces new environments geared towards reinforcement learning and simulations, with significant improvements in speed and compatibility.

Together, these libraries enhance the development and deployment of AI models, catering to both advanced and entry-level users.

- TensorFlow 3.0 Release: November 2024
- PyTorch Lightning 2.5 Release: November 2024
- FastAI 3.0 Release: November 2024
- Hugging Face Transformers 5.0 Release: November 2024
- OpenAI Gym 2.0 Release: Early December 2024

- TensorFlow 3.0 focuses on distributed training and mobile device support.
- PyTorch Lightning 2.5 simplifies model training with new APIs.
- FastAI 3.0 enhances accessibility for beginners with state-of-the-art models.
- Hugging Face Transformers 5.0 improves model efficiency and fine-tuning.
- OpenAI Gym 2.0 adds new reinforcement learning environments.